In [1]:
from datetime import datetime
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def time_translator(time):
    # translate time(yyyy-mm-ddThh:mm:ss) into time(sec)
    return int(datetime.strptime(time, "%Y-%m-%dT%H:%M:%S").timestamp())

def time_to_seconds(time):
    # translate time(hh:mm:ss) into time(sec)
    t = datetime.strptime(time, "%H:%M:%S")
    return t.hour * 3600 + t.minute * 60 + t.second

def trans_x_time(df, unit, unittime, trans_firsttime):
    # cpu_use_rate.index * unittime + trans_firsttime
    # translate df.index(time block) into time(%Y-%m-%dT%H:%M:%S)
    time = [datetime.fromtimestamp(i * unittime + trans_firsttime).strftime('%Y-%m-%dT%H:%M:%S') for i in range(unit)]
    df.index = time
    

def init_the_time(firsttime, lasttime, unit):
    # set the time data, return trans_firsttime, trans_lasttime, unittime
    trans_lasttime = time_translator(lasttime)
    trans_firsttime = time_translator(firsttime)
    unittime = (trans_lasttime - trans_firsttime) // unit
    return trans_firsttime, trans_lasttime, unittime

def extend_node_list(nodelist):
    # icpnp[101-103] -> [icpnp101, icpnp102, icpnp103]
    # 使用正規表達式來解析字串
    pattern = r'([a-zA-Z]+)(\[\d+-\d+(?:,\d+-\d+)*\])'
    matches = re.findall(pattern, nodelist)

    # 將符合的結果整理成 list
    result = []
    for match in matches:
        ranges = match[1][1:-1].split(',')
        for r in ranges:
            start, end = map(int, r.split('-'))
            for i in range(start, end + 1):
                result.append(f"{match[0]}{i}")
    return result

def job_processor(row, trans_firsttime, unittime):
    """
    Process the job imformation into:
        jobstart (int):        *Begin unit* of the job
        jobend (int):          *End unit* of the job
        nodelist (list):       All nodes the job works
        allocated_cpu (int):   Number of allocated CPU
        cpu_useratio (float):  Utilization of CPU
        wait_time (int):       Waiting time of the job
    
    Parameters:
        row (DataFrame):       One job imformation from dataframe
        trans_firsttime (int): Begin time of timeline(sec)
        unittime (int):        Length of one unit time(sec)
    """
    
    try:
        jobstart = (time_translator(row.Start) - trans_firsttime) // unittime
    except:
        jobstart = 0
    try:
        jobend = (time_translator(row.End) - trans_firsttime) // unittime
    except:
        jobend = 0

    nodelist = row.NodeList
    if '[' in nodelist: nodelist = extend_node_list(nodelist)
    result = [jobstart, jobend, nodelist]
    
    # allocated_cpu part
    try:
        allocated_cpu = int(row.NCPUS) / int(row.NNodes)
        result.append(allocated_cpu)
    except:
        allocated_cpu = 0
        result.append(allocated_cpu)
    
    # cpu_useratio part
    try:
        cpu_useratio = time_to_seconds(row.TotalCPU) / (time_translator(row.End) - time_translator(row.Start)) / int(row.NCPUS)
        result.append(cpu_useratio)
    except:
        cpu_useratio = 0
        result.append(cpu_useratio)
    
    # wait_time part
    try:
        wait_time = time_translator(row.Start) - time_translator(row.Submit)
        result.append(wait_time)
    except:
        wait_time = None
        result.append(wait_time)

    #NCPUS
    result.append(row.NCPUS)
    
    #work time
    try:
        work_time = time_translator(row.End) - time_translator(row.Start)
        result.append(work_time)
    except:
        work_time = None
        result.append(work_time)
        
    # submit_time

    # max memory(rss)
    

    return result

def make_data():
    # Creat the zero matrix
    data = np.zeros((300, 598))
    # Translate into dataFrame
    data = pd.DataFrame(data)
    # Set the node name
    columns = []
    for i in range(101, 157):
        columns.append(f'icpnq{i}')
    for i in range(201, 257):
        columns.append(f'icpnq{i}')
    for i in range(301, 357):
        columns.append(f'icpnq{i}')
    for i in range(401, 457):
        columns.append(f'icpnq{i}')
    for i in range(501, 557):
        columns.append(f'icpnq{i}')
    for i in range(601, 657):
        columns.append(f'icpnq{i}')
    for i in range(701, 757):
        columns.append(f'icpnq{i}')
    for i in range(101, 157):
        columns.append(f'icpnp{i}')
    for i in range(201, 257):
        columns.append(f'icpnp{i}')
    for i in range(301, 349):
        columns.append(f'icpnp{i}')
    for i in range(1, 7):
        columns.append(f'gpn0{i}')
    for i in range(1, 41):
        columns.append(f'ncpn{i}')
    # Renew column name of dataFrame
    data.columns = columns
    return data

def add_value_to_data(data, timestart, timeend, nodelist, value):
    """
    Add value on data[timestart:timeend+1, nodelist]
    
    Parameters:
        data (DataFrame): DataFrame to catch the value(from make_data())
        timestart (int):  Begin time of the job
        timeend (int):    End time of the job
        nodelist (list):  Node list of the job
        value (float):    value you want to add
    """
    data.loc[timestart:timeend+1, nodelist] += value
    return data

def plot_heatmap(df, title):
    # Draw heatmap
    fig = go.Figure(data=go.Heatmap(
        z=df.values,
        x=df.columns,
        y=df.index,
        colorscale=[[0, 'rgb(255,255,255)'], [0.0001, 'rgb(200,200,255)'], [1, 'rgb(0,0,255)']],
        showscale=True,
        colorbar=dict(thickness=20, ticklen=4),
        zmin=0,  # 最小值為0
        zmax=df.values.max()
    ))

    fig.update_layout(
        title=title,
        xaxis_nticks=36,
        plot_bgcolor='White',  # 將背景設置為白色
        #width=500,  # 圖的寬度
        height=1500,
        yaxis=dict(tickfont=dict(size=10)),  # 調整y軸標籤字體大小
        xaxis=dict(tickfont=dict(size=10))  # 調整x軸標籤字體大小
    )

    fig.update_traces(hoverongaps=False)  # 不顯示空值的tooltip
    fig.update_traces(zmid=0, colorbar=dict(
        tickvals=[0, df.values.max()],
        ticktext=['0', str(df.values.max())]
    ))

    fig.add_shape(
        type='line', line=dict(dash='dash'),
        #%Y-%m-%dT%H:%M:%S
        x0 = "2024-06-24T00:00:00",
        x1 = "2024-06-24T00:00:00",
        y0 = "icpnq101",
        y1 = "ncpn40",
        name='2024-06-24'
    )

    fig.add_shape(
        type='line', line=dict(dash='dash'),
        #%Y-%m-%dT%H:%M:%S
        x0 = "2024-04-20T03:57:00",
        x1 = "2024-04-20T03:57:00",
        y0 = "icpnq101",
        y1 = "ncpn40",
        name='2024-04-20'
    )

    fig.add_shape(
        type='line', line=dict(dash='solid'),
        #%Y-%m-%dT%H:%M:%S
        x0 = "2024-01-01T00:00:00",
        x1 = "2024-07-01T00:00:00",
        y0 = "icpnp101",
        y1 = "icpnp101"
    )

    fig.add_shape(
        type='line', line=dict(dash='solid'),
        #%Y-%m-%dT%H:%M:%S
        x0 = "2024-01-01T00:00:00",
        x1 = "2024-07-01T00:00:00",
        y0 = "icpnq256",
        y1 = "icpnq256"
    )

    fig.show()

def plot_bar(data, x_title, y_title, title):
    
    df = pd.DataFrame({'id':[i for i in range(len(data))], 'wait':[time/3600 for time in data]})
    fig = px.bar(df, x='id', y='wait')
    fig.update_layout(
        title=title,
        xaxis_title=x_title,
        yaxis_title=y_title
    )
    fig.update_traces(marker_color='rgb(0, 0, 0)', marker_line_color='rgb(0, 0, 0)',
                      marker_line_width=1.5, opacity=0.6)
    
    fig.show()

def main():
    """
    Load dataframe and time setting -> Read row and calculate the value -> Draw pictures with value.
    You can set the dataframe(log file) path, time setting and workflow in main function.

    *If you want to analyze more information from the dataframe, it is easy to achieve by modifying the job_processor() and workflow in the main().*

    Parameters:
        log (DataFrame):  Target dataFrame
        firsttime (date): Begin of the data (yyyy-mm-ddThh:mm:ss)
        unit (int):       Number you want to cut the timeline (Resolution)
        NODEMAX (int):    Number of nodes
    """
    # File path and time setting
    log = pd.read_parquet('20240702_F1_log.parquet')
    log = log[log.Submit < '2024-07-01T00:00:00']
    firsttime = '2024-01-01T00:00:00'
    unit = 300
    NODEMAX = 598
    normal_job = 0 # see only not fail jobs or not (0 = all jobs)

    # Prepare dataframe for catching the value from job_processor
    # more picture -> more make_data()
    lasttime = log.query('End != "Unknown"').End.sort_values(ascending=0).iloc[0]
    trans_firsttime, trans_lasttime, unittime = init_the_time(firsttime, lasttime, unit)
    cpu_use_rate = make_data()
    cpu_occupy = make_data()
    wait_time = []
    work_time = []

    
    for index, row in log.iterrows():
        if normal_job:
            if row.State not in ['FAILED', 'TIMEOUT']:
                result = job_processor(row, trans_firsttime, unittime)
                # result = [jobstart(int), jobend(int), nodelist(list), allocated_cpu(int), cpu_useratio(float), wait_time(int), NCPUS, work_time(int)]
                try:
                    # add_value_to_data(data, timestart, timeend, nodelist, value)
                    cpu_use_rate = add_value_to_data(cpu_use_rate, result[0], result[1], result[2], result[4])
                except:
                    pass

                try:
                    # add_value_to_data(data, timestart, timeend, nodelist, value)
                    cpu_occupy = add_value_to_data(cpu_occupy, result[0], result[1], result[2], result[3])
                except:
                    pass
                if result[5] != None:
                    wait_time.append(result[5])
        
        else:
            result = job_processor(row, trans_firsttime, unittime)
            # result = [jobstart(int), jobend(int), nodelist(list), allocated_cpu(int), cpu_useratio(float), wait_time(int), NCPUS, work_time(int)]
            try:
                # add_value_to_data(data, timestart, timeend, nodelist, value)
                pass
                #cpu_use_rate = add_value_to_data(cpu_use_rate, result[0], result[1], result[2], result[4])
            except:
                pass

            try:
                # add_value_to_data(data, timestart, timeend, nodelist, value)
                cpu_occupy = add_value_to_data(cpu_occupy, result[0], result[1], result[2], result[3])
            except:
                pass
            
            # wait time
            if result[5] != None:
                wait_time.append([int(result[6]), int(result[5])])
                
            #work time
            if result[7] != None:
                work_time.append([int(result[6]), result[7]])

    #transform the x-axis of dfs ( unit -> yyyy-mm-ddThh:mm:ss )
    #trans_x_time(cpu_use_rate.index * unittime + trans_firsttime)
    #trans_x_time(cpu_use_rate, unit, unittime, trans_firsttime)
    trans_x_time(cpu_occupy, unit, unittime, trans_firsttime)

    return cpu_use_rate, cpu_occupy, wait_time, work_time





if __name__ == '__main__':
    cpu_use_rate, cpu_occupy, wait_time, work_time = main()
    #plot_heatmap(cpu_use_rate.T, 'Utilization of CPU (Not fail job)')
    plot_heatmap(cpu_occupy.T, 'Allocated CPU (All jobs)')
    #plot_bar(wait_time, 'Allocated CPU ( /job )', 'Waiting time ( hr )', 'Waiting Time Distribution')
    pass


In [3]:
alloc_cpu = []
how_long_it_wait = []
for pair in wait_time:
    alloc_cpu.append(pair[0])
    how_long_it_wait.append(pair[1])

waiting_time = pd.DataFrame({'#CPU':alloc_cpu, 'Waiting Time':how_long_it_wait})

In [9]:
test = waiting_time.groupby('#CPU')

In [10]:
test.describe()

Waiting Time                                                         
             count        mean          std  min  25%  50%    75%       max
#CPU                                                                       
1           6466.0   39.035571   803.054507  0.0  0.0  0.0    0.0   27736.0
2           1316.0  238.509878  6786.965510  0.0  0.0  0.0    0.0  243051.0
3            256.0    1.632812    18.109199  0.0  0.0  0.0    0.0     283.0
4          38856.0   25.170450   551.396730  0.0  0.0  0.0    0.0   36581.0
5            310.0    0.612903     3.561548  0.0  0.0  0.0    0.0      30.0
...            ...         ...          ...  ...  ...  ...    ...       ...
58800          1.0    0.000000          NaN  0.0  0.0  0.0    0.0       0.0
59584          3.0    0.333333     0.577350  0.0  0.0  0.0    0.5       1.0
60368         10.0    0.000000     0.000000  0.0  0.0  0.0    0.0       0.0
60480         28.0  357.035714   643.798305  0.0  0.0  0.5  364.0    2011.0
61600         11.0    0.363636     0.504525  0.0  0.0  0.0    1.0       1.0

[567 rows x 8 columns]

In [11]:
df = pd.DataFrame({'#CPU':[i for i in waiting_time.groupby('#CPU').describe().index], 'wait time':[time for time in waiting_time.groupby('#CPU').describe().iloc[:,1]]})
fig = px.bar(df, x='#CPU', y='wait time')
fig.update_layout(
    title='#CPU v.s. wait time',
    xaxis_title="#CPU",
    yaxis_title='wait time (second)'
    )
fig.update_traces(marker_color='rgb(0, 0, 0)', marker_line_color='rgb(0, 0, 0)',
                    marker_line_width=1.5, opacity=0.6)
    
fig.show()

In [8]:
df = pd.DataFrame({'#CPU':[i for i in waiting_time.groupby('#CPU').describe().index], 'count':[count for count in waiting_time.groupby('#CPU').describe().iloc[:,0]]})
fig = px.bar(df, x='#CPU', y='count', log_y=True)
fig.update_layout(
    title='#CPU v.s. count',
    xaxis_title="#CPU",
    yaxis_title='count'
    )
fig.update_traces(marker_color='rgb(0, 0, 0)', marker_line_color='rgb(0, 0, 0)',
                    marker_line_width=1.5, opacity=0.6)
    
fig.show()

In [4]:
df = pd.DataFrame({'#CPU':[i for i in waiting_time.groupby('#CPU').describe().index], 'count':[count for count in waiting_time.groupby('#CPU').describe().iloc[:,0]]})
fig = px.bar(df, x='#CPU', y='count')
fig.update_layout(
    title='#CPU v.s. count',
    xaxis_title="#CPU",
    yaxis_title='count'
    )
fig.update_traces(marker_color='rgb(0, 0, 0)', marker_line_color='rgb(0, 0, 0)',
                    marker_line_width=1.5, opacity=0.6)
    
fig.show()

In [20]:
i = 0
cumulative = []
for c in df.iloc[:,1]:
    i += c
    cumulative.append(i)
df["cumulative"] = cumulative

fig = go.Figure()
fig_area = px.area(x=df.iloc[:,0], y=df.iloc[:,2])
for trace in fig_area.data:
    fig.add_trace(trace)
fig.add_trace(go.Bar(x=df.iloc[:,0], y=df.iloc[:,2]))
#fig.add_trace(go.Area(x=df.iloc[:,0], y=df.iloc[:,2]))
#fig = px.area(df, x='#CPU', y='cumulative')
#fig = px.bar(df, x='#CPU', y='cumulative')

fig.update_layout(
    title='#CPU v.s. cumulative count',
    xaxis_title="#CPU",
    yaxis_title='cumulative count'
    )
fig.update_traces(marker_color='rgb(0, 0, 0)', marker_line_color='rgb(0, 0, 0)',
                    marker_line_width=1.5, opacity=0.6)

fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0 = 0,
        x1 = df.iloc[-1,0],
        y0 = i*0.95,
        y1 = i*0.95
    )
    
fig.show()

In [64]:
alloc_cpu = []
how_long_it_work = []
for pair in work_time:
    alloc_cpu.append(pair[0])
    how_long_it_work.append(pair[1])

working_time = pd.DataFrame({'#CPU':alloc_cpu, 'Working Time':how_long_it_work})

In [65]:
working_time.groupby('#CPU').describe()

Working Time                                                         \
             count         mean           std    min    25%    50%    75%   
#CPU                                                                        
1           6466.0  3798.608877  20558.531702    0.0    3.0   45.0  161.0   
2           1316.0  3851.034954  21201.791788    0.0    1.0   63.0  780.5   
3            256.0  5466.921875  26602.089964    0.0    1.0    4.0   92.5   
4          38856.0   142.675983   3669.085272    0.0    4.0    6.0   10.0   
5            310.0  2674.896774  15297.751103    0.0    2.0    3.0   76.0   
...            ...          ...           ...    ...    ...    ...    ...   
58800          1.0     0.000000           NaN    0.0    0.0    0.0    0.0   
59584          3.0     1.666667      2.886751    0.0    0.0    0.0    2.5   
60368         10.0   279.100000     74.403480  237.0  243.0  245.0  248.5   
60480         28.0   245.714286    155.284889    0.0  240.0  243.5  297.0   
61600         11.0   211.272727    311.095834    5.0   11.0   93.0  226.5   

                 
            max  
#CPU             
1      343302.0  
2      345606.0  
3      172827.0  
4      345619.0  
5      154390.0  
...         ...  
58800       0.0  
59584       5.0  
60368     427.0  
60480     566.0  
61600     818.0  

[566 rows x 8 columns]

In [66]:
df = pd.DataFrame({'#CPU':[i for i in working_time.groupby('#CPU').describe().index], 'work time':[time for time in working_time.groupby('#CPU').describe().iloc[:,1]]})
fig = px.bar(df, x='#CPU', y='work time')
fig.update_layout(
    title='#CPU v.s. work time',
    xaxis_title="#CPU",
    yaxis_title='work time (second)'
    )
fig.update_traces(marker_color='rgb(0, 0, 0)', marker_line_color='rgb(0, 0, 0)',
                    marker_line_width=1.5, opacity=0.6)
    
fig.show()

In [1]:
import dask.dataframe as dd

log = dd.read_parquet('./20240702_F1_log.parquet')

,Account,AllocCPUS,AllocNodes,AllocTRES,AssocID,CPUTimeRAW,ElapsedRaw,Eligible,End,ExitCode,...,Submit,Suspended,SystemCPU,SystemComment,TimelimitRaw,TotalCPU,UID,User,UserCPU,WorkDir
0,root,2,1,"billing=2,cpu=2,mem=515350M,node=1",6,0,0,2024-01-03T11:44:47,2024-01-03T11:44:47,0:0,...,2024-01-03T11:44:47,00:00:00,00:00:00,,UNLIMITED,00:00:00,0,root,00:00:00,/home
1,root,2,1,"billing=2,cpu=2,mem=515350M,node=1",6,0,0,2024-01-03T11:44:47,2024-01-03T11:44:47,0:0,...,2024-01-03T11:44:47,00:00:00,00:00:00,,,00:00:00,,,00:00:00,
2,root,2,1,"cpu=2,mem=515350M,node=1",6,0,0,2024-01-03T11:44:47,2024-01-03T11:44:47,0:0,...,2024-01-03T11:44:47,00:00:00,00:00:00,,,00:00:00,,,00:00:00,
3,root,7,0,,6,0,0,2024-01-03T16:38:08,2024-01-03T17:27:09,0:0,...,2024-01-03T16:38:08,00:00:00,00:00:00,,Partition_Limit,00:00:00,0,root,00:00:00,/root
4,root,7,0,,6,0,0,2024-01-04T10:49:39,2024-01-04T11:40:45,0:0,...,2024-01-04T10:49:39,00:00:00,00:00:00,,Partition_Limit,00:00:00,0,root,00:00:00,/root
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206635,mst110307,1344,12,"billing=1344,cpu=1344,mem=5789952M,node=12",12866,67200,50,2024-04-17T15:46:29,2024-04-17T15:47:38,0:0,...,2024-04-17T15:46:29,00:00:00,22:52.758,,2880,07:07:21,21695,b10606105,06:44:28,/work1/b10606105/pure-electrolyte/F4DEE-DEE/15...
206636,mst110307,112,1,"cpu=112,mem=482496M,node=1",12866,5600,50,2024-04-17T15:46:48,2024-04-17T15:47:38,0:0,...,2024-04-17T15:46:48,00:00:00,00:00.129,,,00:00.207,,,00:00.077,
206637,mst110307,1344,12,"billing=1344,cpu=1344,mem=5789952M,node=12",12866,67200,50,2024-04-17T15:46:48,2024-04-17T15:47:38,0:0,...,2024-04-17T15:46:48,00:00:00,00:00.004,,,00:00.007,,,00:00.003,
206638,mst110307,1344,12,"cpu=1344,mem=5789952M,node=12",12866,61824,46,2024-04-17T15:46:51,2024-04-17T15:47:37,0:0,...,2024-04-17T15:46:51,00:00:00,22:52.623,,,07:07:21,,,06:44:28,


In [26]:
import pandas as pd
import dask.dataframe as dd

d = {'col1': [1, 2, 3, 4], 'col2': [5, 6, 7, 8]}
df = dd.from_pandas(pd.DataFrame(d))
#df = pd.DataFrame(d)

In [29]:
d

{'col1': [1, 2, 3, 4], 'col2': [5, 6, 7, 8]}

In [30]:
for i,r in df.iterrows():
    print(r)
print('-------------')
for i,r in df.iterrows():
    r += 1
    print(r)
print('-------------')
for i,r in df.iterrows():
    r += 1
    print(r)

#dask怪怪的我猜可能是東西留在記憶體裡了所以數值一直累加

col1     8
col2    12
Name: 0, dtype: int64
col1     9
col2    13
Name: 1, dtype: int64
col1    10
col2    14
Name: 2, dtype: int64
col1    11
col2    15
Name: 3, dtype: int64
-------------
col1     9
col2    13
Name: 0, dtype: int64
col1    10
col2    14
Name: 1, dtype: int64
col1    11
col2    15
Name: 2, dtype: int64
col1    12
col2    16
Name: 3, dtype: int64
-------------
col1    10
col2    14
Name: 0, dtype: int64
col1    11
col2    15
Name: 1, dtype: int64
col1    12
col2    16
Name: 2, dtype: int64
col1    13
col2    17
Name: 3, dtype: int64


In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("tutorial.db")
con.close()